In [29]:
import os
import shutil
import numpy as np
import subprocess
import glob
from urllib import urlretrieve
import zipfile

import geopandas as gpd
import fiona
import rasterio
from rasterio import features
from rasterio.features import shapes
from shapely.geometry import mapping, shape
from osgeo import gdal, gdalnumeric, ogr, osr
from gdalconst import *
from PIL import Image, ImageDraw


%matplotlib inline
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

from library.geoprocess import rm_and_mkdir, shp_to_shps, raster_to_rasters, polygonize, union_and_filter, get_countries, split_multi_to_single_poly, merge_shapefiles

#### Create individual shapefiles of each country from shapefile of all countries
* load shapefile of all admin areas / countries as geodataframe
* filter out countries not internationally recognized
* loop through rows of geodataframe and save each row as a country-specific shapefile

In [2]:
# load shapefile of all admin areas / countries as geodataframe
gdf = gpd.read_file('data/geo/countries/countries_nf2.shp'); gdf.head(3)

# filter out countries not internationally recognized
country_filter1 = gdf['WB_A3'] != '-99'
gdf = gdf.drop_duplicates(subset='WB_A3')
gdf = gdf[country_filter1].set_index('WB_A3')

# loop through rows of geodataframe and save each row as a country-specific shapefile in newly created dir
# shp_to_shps('data/geo/countries/shp', gdf)

/Users/peter/anaconda/lib/python2.7/site-packages/geopandas/geodataframe.py:376: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super(GeoDataFrame, self).__getitem__(key)


#### Generate city boundaries
* Clip master raster from 2013 by each country shapefile, creating country-specific rasters
* Use subprocess module to run gdal commands in terminal to do this
* Polygonize each country raster
* Select subset of polygons that have light intensity greater than selected thresh
* Union remaining polygons to get contiguous city boundaries
* Intersect with populated places to eliminate non-key cities
* Save outputs to cities directory

In [3]:
# clip master raster from 2013 by each country shapefile to create country-level rasters
input_tif_path = 'data/geo/images/F182013.v4c_web.stable_lights.avg_vis.tif'
input_shp_dir = 'data/geo/countries/shp'
output_tif_dir = 'data/geo/countries/tif'
countries = [x.encode('UTF-8') for x in gdf.index.values]
# raster_to_rasters(countries, input_tif_path, input_shp_dir, output_tif_dir)

In [4]:
# polygonize rasters and save to target directory
input_tif_dir = 'data/geo/countries/tif'
output_shp_dir = 'data/geo/countries/poly'
# polygonize(input_tif_dir, output_shp_dir, countries)

In [5]:
# filter and union countries, save to target directory
input_dir = 'data/geo/countries/poly'
output_dir = 'data/geo/cities/union'
# union_and_filter(input_dir, output_dir, countries)

In [6]:
# split multi-polygons into polygons
input_dir = 'data/geo/cities/union'
output_dir = 'data/geo/cities/split'
# split_multi_to_single_poly(input_dir, output_dir)

In [7]:
# Merge shapefiles in directory
input_dir = 'data/geo/cities/split'
output_dir = 'data/geo/cities/merge'
output_filename = 'merged.shp'
merge_shapefiles(input_dir, output_dir, output_filename)

ABW.shp
ADO.shp
AFG.shp
AGO.shp
ALB.shp
ARE.shp
ARG.shp
ARM.shp
ASM.shp
ATG.shp
AUS.shp
AUT.shp
AZE.shp
BDI.shp
BEL.shp
BEN.shp
BFA.shp
BGD.shp
BGR.shp
BHR.shp
BHS.shp
BIH.shp
BLR.shp
BLZ.shp
BMU.shp
BOL.shp
BRA.shp
BRB.shp
BRN.shp
BTN.shp
BWA.shp
CAF.shp
CAN.shp
CHE.shp
CHI.shp
CHL.shp
CHN.shp
CIV.shp
CMR.shp
COG.shp
COL.shp
CPV.shp
CRI.shp
CUB.shp
CUW.shp
CYM.shp
CYP.shp
CZE.shp
DEU.shp
DJI.shp
DMA.shp
DNK.shp
DOM.shp
DZA.shp
ECU.shp
EGY.shp
ERI.shp
ESP.shp
EST.shp
ETH.shp
FIN.shp
FJI.shp
FRA.shp
FRO.shp
GAB.shp
GBR.shp
GEO.shp
GHA.shp
GIN.shp
GMB.shp
GNB.shp
GNQ.shp
GRC.shp
GRD.shp
GRL.shp
GTM.shp
GUM.shp
GUY.shp
HKG.shp
HND.shp
HRV.shp
HTI.shp
HUN.shp
IDN.shp
IMY.shp
IND.shp
IRL.shp
IRN.shp
IRQ.shp
ISL.shp
ISR.shp
ITA.shp
JAM.shp
JOR.shp
JPN.shp
KAZ.shp
KEN.shp
KGZ.shp
KHM.shp
KNA.shp
KOR.shp
KSV.shp
KWT.shp
LAO.shp
LBN.shp
LBR.shp
LBY.shp
LCA.shp
LIE.shp
LKA.shp
LSO.shp
LTU.shp
LUX.shp
LVA.shp
MAC.shp
MAF.shp
MAR.shp
MCO.shp
MDA.shp
MDG.shp
MDV.shp
MEX.shp
MHL.shp
MKD.shp
MLI.shp


In [9]:
# set CRS of merged shapefile
input_path = 'data/geo/cities/merge/merged.shp'
gdf_merged = gpd.read_file(input_path)
gdf_merged.crs = {'init': 'epsg:4326', 'no_defs': True}
output_shp_path = os.path.join(output_dir, 'merged_crs.shp')
gdf_merged.to_file(output_shp_path)
gdf_merged.head(3)

,FID,geometry
0,0.0,"POLYGON ((61.90316872520486 30.96225290982025,..."
1,1.0,"POLYGON ((61.79485647729509 31.13732486357805,..."
2,2.0,"POLYGON ((64.06108197202268 31.08730430536153,..."


In [28]:
# save pop_places shapefile into new directory called pop_places
# http://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/cultural/ne_10m_populated_places_simple.zip

output_dir = 'data/geo/cities/pop_places' # copied pop places and merged shapefiles into same dir
rm_and_mkdir(output_dir)
zip_filename = 'pops.zip'
zip_path = os.path.join(output_dir, zip_filename)
url = 'http://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/cultural/ne_10m_populated_places_simple.zip'
req = urlretrieve(url, 'data/geo/cities/pop_places/pops.zip')
zip_ref = zipfile.ZipFile(zip_path, 'r')
zip_ref.extractall(directory_to_extract_to)
zip_ref.close()

In [17]:
# intersect populated places with shp
polygons_collection = fiona.open('data/geo/cities/merge/merged_crs.shp')
polygons_next = polygons_collection.next()
polygons = shape(polygons_next['geometry'])

points_collection = fiona.open('data/geo/cities/pop_places/pop_places.shp')
points_next = points_collection.next()
points = shape(points_next['geometry'])

In [21]:

output_shp_path = os.path.join(output_dir, 'merged_crs.shp')
gdf_merged.to_file(output_shp_path) # copy merged_crs.shp to pop_places dir

from osgeo import ogr
ogr.UseExceptions()
ogr_ds = ogr.Open('data/geo/cities/pop_places', True)
SQL = """\
    SELECT ST_Intersection(A.geometry, B.geometry) AS geometry, A.*, B.*
    FROM merged_crs, pop_places B
    WHERE ST_Intersects(A.geometry, B.geometry);
"""
layer = ogr_ds.ExecuteSQL(SQL, dialect='SQLITE')
# copy result back to datasource as a new shapefile
layer2 = ogr_ds.CopyLayer(layer, 'h1_buf_int_ct3')
# save, close
layer = layer2 = ogr_ds = None

RuntimeError: In ExecuteSQL(): sqlite3_prepare(    SELECT ST_Intersection(A.geometry, B.geometry) AS geometry, A.*, B.*
    FROM nyct2010 A, h1_buf B
    WHERE ST_Intersects(A.geometry, B.geometry);
):
  no such table: nyct2010